In [52]:
pip install dspy-ai

You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.

In [53]:
pip install --upgrade jupyter

You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.

## Set up basic evaluation

In [54]:
def read_text_from_file(file_path):
    """
    Reads and returns the content of a text file specified by the file path.
    
    Args:
    file_path (str): The path to the text file.
    
    Returns:
    str: The content of the file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

### Test grep commands

In [55]:
import subprocess

def run_grep_command(command):
    try:
        # Split the command into a list for subprocess
        command_list = command.split()

        # Run the command
        result = subprocess.run(command_list, capture_output=True, text=True, check=True)

        # Save the output to a variable
        output = result.stdout
        
        return output

    except subprocess.CalledProcessError as e:
        print(f"Command '{e.cmd}' returned non-zero exit status {e.returncode}.")
        print(f"Error output: {e.stderr}")
        return None

Now for a simple test

In [56]:
grep_command = "grep -i 'fruit' ./check_agent/data/sample_input/code.txt"
output = run_grep_command(grep_command)

if output:
    print("Grep command output:")
    print(output)
else:
    print("No output or an error occurred.")

Grep command output:
fruit = []
fruit[0] = 'apple'
def fruit():



Create a list of DSPy Example objects

In [57]:
import os
def extract_file_just_path(file_path):
    file_name = os.path.basename(file_path)
    new_path =  os.path.join('./check_agent/data/sample_input/',file_name)
    return new_path

test_path = "/tmp/calue/code.txt"

extracted = extract_file_just_path(test_path)

extracted

'./check_agent/data/sample_input/code.txt'

Dynamically modify file names

In [58]:
def update_grep_command(command):
    parts = command.split()
    updated_parts = [parts[0]]  # Start with the command itself, which is 'grep'
    
    i = 1  # Start processing after 'grep'
    while i < len(parts):
        part = parts[i]
        if part.startswith('-'):
            if i + 1 < len(parts) and not parts[i + 1].startswith('-'):
                # Next part is a file path and current part is an option
                # Check if the part is a standalone option or concatenated with its parameter
                if len(part) > 2 and part[2].isalpha():
                    # Option is concatenated with parameter (like -Ff)
                    updated_parts.append(part)
                else:
                    updated_parts.append(part)  # Add the option
                    updated_parts.append(extract_file_just_path(parts[i + 1]))  # Add the updated path
                    i += 1  # Move past the file path part
            else:
                # It's just an option without following file path (or followed by another option)
                updated_parts.append(part)
        else:
            # This is a standalone file path
            updated_parts.append(extract_file_just_path(part))
        i += 1

    updated_command = ' '.join(updated_parts)
    return updated_command

generated_command = "grep -Ff terms.txt sample.txt code.txt"
updated_command = update_grep_command(generated_command)

print(run_grep_command(updated_command))

print(f"Original command: {generated_command}")
print(f"Updated command: {updated_command}")


./check_agent/data/sample_input/sample.txt:How are you
./check_agent/data/sample_input/sample.txt:mango
./check_agent/data/sample_input/sample.txt:Much ado about nothing
./check_agent/data/sample_input/sample.txt:Adios amigo
./check_agent/data/sample_input/code.txt:fruit[0] = 'apple'

Original command: grep -Ff terms.txt sample.txt code.txt
Updated command: grep -Ff ./check_agent/data/sample_input/terms.txt ./check_agent/data/sample_input/sample.txt ./check_agent/data/sample_input/code.txt


In [59]:
import json
import dspy

def extract_fields(json_file):
    # Read the JSON data from the file
    examples_list = []
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Iterate over each item in the JSON data
    for key, value in data.items():
        question = value.get('question', '')
        op_file = value.get('op_file', '')
        ref_solution = value.get('ref_solution', [''])[0]  # Get the first element of ref_solution list

        example = dspy.Example(question=question, op_file=op_file, answer=ref_solution).with_inputs(question)
        examples_list.append(example)
    return examples_list

# Example usage
json_file = './check_agent/data/questions.json'
questions_dict = extract_fields(json_file)
questions_dict


[Example({'question': 'Display lines containing `an` from the `sample.txt` input file.', 'op_file': 'banana\nmango\n', 'answer': "grep 'an' sample.txt"}) (input_keys={'Display lines containing `an` from the `sample.txt` input file.'}),
 Example({'question': 'Display lines containing `do` as a whole word from the `sample.txt` input file.', 'op_file': 'Just do-it\n', 'answer': "grep -w 'do' sample.txt"}) (input_keys={'Display lines containing `do` as a whole word from the `sample.txt` input file.'}),
 Example({'question': 'Display lines from `sample.txt` that satisfy both of these conditions:\n\n* `he` matched irrespective of case\n* either `World` or `Hi` matched case sensitively', 'op_file': 'Hello World\nHi there\n', 'answer': "grep -i 'he' sample.txt | grep -e 'World' -e 'Hi'"}) (input_keys={'Display lines from `sample.txt` that satisfy both of these conditions:\n\n* `he` matched irrespective of case\n* either `World` or `Hi` matched case sensitively'}),
 Example({'question': 'Displa

### Use DSPy with Typed predictor

In [60]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.

You should consider upgrading via the 'C:\Users\dbake\Documents\GREP_Boi\CS541_CS441_AI_Final_Project\SyntheticData\synth-env\Scripts\python.exe -m pip install --upgrade pip' command.

In [61]:
import dspy
from dotenv import load_dotenv

load_dotenv("../.env")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gemini_flash = dspy.Google(model="gemini-1.5-flash-latest", temperature=0)

dspy.settings.configure(lm=gemini_flash)


#### Create simple signature

In [62]:
from pydantic import BaseModel, Field, conlist
from typing import List, NamedTuple

class GrepPair(NamedTuple):
    command: str
    description: str

class Output(BaseModel):
    commands: conlist(GrepPair, min_length=10, max_length=10) = Field(..., description="List of 10 pairs of grep commands and a natural language description ")

class CommentInput(BaseModel):
    commands: conlist(str, min_length=10, max_length=10) = Field(..., description="List of 10 grep commands")

class CommentOutput(BaseModel):
    descriptions: conlist(str, min_length=10, max_length=10) = Field(..., description="corresponding list of natural language descriptions for the grep commands")

class test_generation(dspy.Signature):
    """Given the contents of a file and the file name as input create a useful and interesting grep command"""
    file = dspy.InputField(desc="The file to make grep commands for")
    filename = dspy.InputField(desc="The filename")
    command: Output = dspy.OutputField()

generate_answer = dspy.TypedPredictor(test_generation)

code_file = read_text_from_file("./check_agent/data/sample_input/code.txt")

resp = generate_answer(file=code_file, filename="code.txt")

the_command=resp.command

print(resp.command)

commands=[GrepPair(command="grep 'fruit' code.txt", description="Find all lines in code.txt that contain the word 'fruit'."), GrepPair(command="grep -E '^[a-zA-Z]+$' code.txt", description='Find all lines in code.txt that contain only letters.'), GrepPair(command="grep -v 'print' code.txt", description="Find all lines in code.txt that do not contain the word 'print'."), GrepPair(command="grep -r 'def' .", description="Recursively search for lines containing 'def' in all files within the current directory."), GrepPair(command="grep -n 'banana' code.txt", description="Find all lines containing 'banana' in code.txt and display the line number."), GrepPair(command="grep -c 'apple' code.txt", description="Count the number of lines in code.txt that contain 'apple'."), GrepPair(command="grep -w 'fruit' code.txt", description="Find all lines in code.txt that contain 'fruit' as a whole word."), GrepPair(command="grep -A 2 'def' code.txt", description="Find all lines containing 'def' in code.txt

In [63]:
print(resp)

Prediction(
    command=Output(commands=[GrepPair(command="grep 'fruit' code.txt", description="Find all lines in code.txt that contain the word 'fruit'."), GrepPair(command="grep -E '^[a-zA-Z]+$' code.txt", description='Find all lines in code.txt that contain only letters.'), GrepPair(command="grep -v 'print' code.txt", description="Find all lines in code.txt that do not contain the word 'print'."), GrepPair(command="grep -r 'def' .", description="Recursively search for lines containing 'def' in all files within the current directory."), GrepPair(command="grep -n 'banana' code.txt", description="Find all lines containing 'banana' in code.txt and display the line number."), GrepPair(command="grep -c 'apple' code.txt", description="Count the number of lines in code.txt that contain 'apple'."), GrepPair(command="grep -w 'fruit' code.txt", description="Find all lines in code.txt that contain 'fruit' as a whole word."), GrepPair(command="grep -A 2 'def' code.txt", description="Find all lin

In [64]:
print(code_file)

fruit = []
fruit[0] = 'apple'
def fruit():
    print('banana')

